In [9]:
!pip install --upgrade pytorch_lightning

     -------------------------------------- 585.5/585.5 KB 6.1 MB/s eta 0:00:00
     -------------------------------------- 140.6/140.6 KB 8.1 MB/s eta 0:00:00
     -------------------------------------- 418.2/418.2 KB 8.7 MB/s eta 0:00:00
     -------------------------------------- 155.4/155.4 KB 9.1 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB ? eta 0:00:00
     -------------------------------------- 556.0/556.0 KB 7.0 MB/s eta 0:00:00

ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You should consider upgrading via the 'C:\dev\anaconda3\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 122.2/122.2 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:
      Successfully uninstalled typing-extensions-3.7.4.2
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.47.0
    Uninstalling tqdm-4.47.0:
      Successfully uninstalled tqdm-4.47.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1


In [10]:
!pip show pytorch_lightning

Name: pytorch-lightning
Version: 0.8.5
Summary: PyTorch Lightning is the lightweight PyTorch wrapper for ML researchers. Scale your models. Write less boilerplate.
Home-page: https://github.com/PyTorchLightning/pytorch-lightning
Author: William Falcon et al.
Author-email: waf2107@columbia.edu
License: Apache-2.0
Location: c:\dev\anaconda3\lib\site-packages
Requires: future, numpy, PyYAML, tensorboard, torch, tqdm
Required-by: 


In [6]:
import torchtext.data

In [4]:
import torch
import torchvision
import pytorch_lightning as pl
from transforms import *
import numpy as np

ImportError: cannot import name 'Batch' from 'torchtext.data' (C:\dev\anaconda3\lib\site-packages\torchtext\data\__init__.py)

In [32]:
class PretrainCIFARDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.picture_ids = None
        self.train_dataset = None
        self.data_dir = "datasets"
        self.download = True
        self.batch_size = 256
        self.num_workers = 5
        self.num_labeled_classes = 80
        self.num_unlabeled_classes = 20
        self.dataset_class = getattr(torchvision.datasets, "CIFAR100")
        self.transform_train = get_transforms("unsupervised", "CIFAR100")
        self.transform_val = get_transforms("eval", "CIFAR100")

    def setup(self, stage=None):
        self.picture_ids = [ [] for _ in range(100) ] #np.zeros([100,1])

        # train dataset
        self.train_dataset = self.dataset_class(
            self.data_dir, train=True, transform=self.transform_train, download=self.download
        )

        for data in self.train_dataset:
            self.picture_ids[data[1]].append(data[1])


In [43]:
dm = PretrainCIFARDataModule()
dm.setup()

Files already downloaded and verified


KeyboardInterrupt: 

In [53]:
import numpy as np
labeled_classes = range(80)

# train dataset
train_dataset = getattr(torchvision.datasets, "CIFAR100")(
    "datasets/", train=True, transform=get_transforms("unsupervised", "CIFAR100"), download=True
)
train_indices_lab = np.where(
    np.isin(np.array(train_dataset.targets), labeled_classes)
)[0]

train_dataset = torch.utils.data.Subset(train_dataset, train_indices_lab)
print(len(train_dataset))

# val datasets
val_dataset = getattr(torchvision.datasets, "CIFAR100")(
    "datasets/", train=False, transform=get_transforms("eval", "CIFAR100"), download=True
)
val_indices_lab = np.where(np.isin(np.array(val_dataset.targets), labeled_classes))[0]
val_dataset = torch.utils.data.Subset(val_dataset, val_indices_lab)
print(len(val_dataset))


Files already downloaded and verified
40000
Files already downloaded and verified
8000


In [106]:
num_labeled_classes = 80
labeled_classes = range(num_labeled_classes)
picture_ids_train = [[] for _ in labeled_classes]
picture_ids_val = [[] for _ in labeled_classes]

# download and load the train dataset
train_dataset = getattr(torchvision.datasets, "CIFAR100")(
    "datasets/", train=True, transform=get_transforms("unsupervised", "CIFAR100"), download=True
)

# download and load the val dataset
val_dataset = getattr(torchvision.datasets, "CIFAR100")(
    "datasets/", train=False, transform=get_transforms("eval", "CIFAR100"), download=True
)

# take only the indices of the first num_labeled_classes classes
train_indices_lab = np.where(
     np.isin(np.array(train_dataset.targets), labeled_classes)
)[0]

# take only the indices of the first num_labeled_classes classes
val_indices_lab = np.where(
     np.isin(np.array(val_dataset.targets), labeled_classes)
)[0]

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [107]:
train_num_samples_per_class = []
val_num_samples_per_class = []

# starting from class 0, perform long-tailed sampling by using a decreasing exponential function
# the number of samples considered per class is stored as a list
for i in range(num_labeled_classes):
    train_num_samples_per_class.append(int(500  * (0.96 ** i)))
    val_num_samples_per_class.append(int(100  * (0.96 ** i)))

print(train_num_samples_per_class)
print(val_num_samples_per_class)

print(len(train_num_samples_per_class))
print(len(val_num_samples_per_class))

[500, 480, 460, 442, 424, 407, 391, 375, 360, 346, 332, 319, 306, 294, 282, 271, 260, 249, 239, 230, 221, 212, 203, 195, 187, 180, 172, 166, 159, 153, 146, 141, 135, 129, 124, 119, 115, 110, 105, 101, 97, 93, 90, 86, 82, 79, 76, 73, 70, 67, 64, 62, 59, 57, 55, 52, 50, 48, 46, 44, 43, 41, 39, 38, 36, 35, 33, 32, 31, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19]
[100, 96, 92, 88, 84, 81, 78, 75, 72, 69, 66, 63, 61, 58, 56, 54, 52, 49, 47, 46, 44, 42, 40, 39, 37, 36, 34, 33, 31, 30, 29, 28, 27, 25, 24, 23, 23, 22, 21, 20, 19, 18, 18, 17, 16, 15, 15, 14, 14, 13, 12, 12, 11, 11, 11, 10, 10, 9, 9, 8, 8, 8, 7, 7, 7, 7, 6, 6, 6, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3]
80
80


In [108]:
# since the elements of the train/val dataset variables are not sorted by the numeric value of the class, we first filter each sample by its class yielding always
# 500 indices for train and 100 samples for val. Then, the long-tailed sampling is performed by taking only as much samples as the value at position i in train/val_num_samples_per_class
# i.e., the first class 0 will have the most samples and the last class will have the least samples
# the Subset class is then used to take the corresponding samples from the dataset and at the same time drop the remaining classes from the dataset

index_list_final_train = []
index_list_final_val = []
for i in labeled_classes:

    index_list_final_train += np.where(np.isin(np.array(train_dataset.targets), [i]))[0][:train_num_samples_per_class[i]].tolist()
    index_list_final_val += np.where(np.isin(np.array(val_dataset.targets), [i]))[0][:val_num_samples_per_class[i]].tolist()

train_dataset = torch.utils.data.Subset(train_dataset, index_list_final_train)
val_dataset = torch.utils.data.Subset(val_dataset, index_list_final_val)

11982
2367
11982
2367
